In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import reportParser
import pandas as pd
from isotherm import Isotherm
matplotlib.use('TkAgg')

In [2]:
data = Isotherm()
data.load_kernel()

In [3]:
raw_data = reportParser.get_isotherm_and_distribution("test_report.txt")
isotherm_data = reportParser.get_numpy_arrays(raw_data)

data.isothermData = isotherm_data["adsorption"]
data.pressureData = isotherm_data["p_adsorption"]
data.interpolate_isotherm()
data.process_kernel()

C:\Users\anton\PycharmProjects\isotherm-analyzer\tools\reportParser.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(io.StringIO(value), sep="\s+|\t+|\s+\t+|\t+\s+")
C:\Users\anton\PycharmProjects\isotherm-analyzer\tools\reportParser.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(io.StringIO(value), sep="\s+|\t+|\s+\t+|\t+\s+")
C:\Users\anton\PycharmProjects\isotherm-analyzer\tools\reportParser.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are

In [4]:
alpha = 0
beta = 1

In [5]:
data.process_isotherm(alpha, beta)

C:\Users\anton\PycharmProjects\isotherm-analyzer\inverse.py:18: RuntimeWarning: divide by zero encountered in log
  adsorption)).sum(axis=0) + alpha * np.sum(pore_dist * np.log(pore_dist)) +
C:\Users\anton\PycharmProjects\isotherm-analyzer\inverse.py:18: RuntimeWarning: invalid value encountered in multiply
  adsorption)).sum(axis=0) + alpha * np.sum(pore_dist * np.log(pore_dist)) +


In [6]:
# plt.plot(data.kernel_a_array, data.defined_PSD, marker=".")
# plt.show()

In [7]:
def calculate_error(isotherm: Isotherm):
    defined_error = np.sum(np.abs(
    isotherm.restored_isotherm - isotherm.interpolated_isotherm) / isotherm.interpolated_isotherm) / len(
    isotherm.interpolated_isotherm)
    return defined_error

def calculate_isotherm_error(data):
    return np.linalg.norm(np.subtract(data.restored_isotherm, data.interpolated_isotherm)).sum(axis=0)**2

def calculate_alpha_regularisation_error(data):
    return np.sum(data.defined_PSD * np.log(data.defined_PSD))

def calculate_beta_regularisation_error(data):
    return np.sum(np.square(np.diff(data.defined_PSD)/data.kernel_a_array[:-1]))

def calculate_tikhonov_regularisation_error(data):
    return np.linalg.norm(data.defined_PSD)**2

In [8]:
def find_optimal_regularization(isotherm_errors, regularisation_errors):
    def distance_point_to_line(p, p1, p2):
        return (np.abs((p2[1] - p1[1]) *
                      p[0] - (p2[0] - p1[0]) * p[1] + p2[0] * p1[1] - p2[1] * p1[0])
                / np.sqrt((p2[1] - p1[1])**2 + (p2[0] - p1[0])**2))
    def find_knee(x, y):
        # x: координаты по оси X (норма решения)
        # y: координаты по оси Y (ошибка приближения)
        p1 = np.array([x[0], y[0]])
        p2 = np.array([x[-1], y[-1]])
        
        distances = np.array([distance_point_to_line(np.array([x[i], y[i]]), p1, p2) for i in range(len(x))])
        
        knee_index = np.argmax(distances)
        
        return knee_index
    return find_knee(isotherm_errors, regularisation_errors)

In [9]:
def calculate_L_curve(data: Isotherm, beta_array):
    isotherm_errors = []
    regularisation_errors = []
    for beta in beta_array:
        data.process_isotherm(beta, 0, regularization="tikhonov")
        isotherm_errors.append(calculate_isotherm_error(data))#(calculate_isotherm_error(data))
        #regularisation_errors.append(calculate_alpha_regularisation_error(data))
        #regularisation_errors.append(calculate_beta_regularisation_error(data))
        regularisation_errors.append(calculate_tikhonov_regularisation_error(data))
    return isotherm_errors, regularisation_errors

In [10]:
coarse_mesh =  list(np.logspace(-4, -1, 15)) #[0, 0.01, 0.025, 0.25, 1.5, 5]
isotherm_errors, regularisation_errors = calculate_L_curve(data, coarse_mesh)

In [11]:
plt.yscale("log")
plt.xscale("log")
plt.plot(isotherm_errors, regularisation_errors, marker=".")
for i, txt in enumerate(coarse_mesh):
    plt.annotate(str(round(float(txt), 3)), (isotherm_errors[i], regularisation_errors[i]))
plt.show()

In [12]:
coarse_mesh_idx = find_optimal_regularization(np.log(isotherm_errors), np.log(regularisation_errors))
print(coarse_mesh[coarse_mesh_idx], coarse_mesh_idx)
# print(coarse_mesh)

0.03727593720314938 12


In [13]:
data.process_isotherm(coarse_mesh[coarse_mesh_idx], 0, regularization="tikhonov")
calculate_error(data)

0.05014431717639259

In [14]:
fine_mesh_len = 10
fine_mesh_1 = list(np.linspace(coarse_mesh[coarse_mesh_idx - 1], coarse_mesh[coarse_mesh_idx], fine_mesh_len))[1:]
fine_mesh_2 = list(np.linspace(coarse_mesh[coarse_mesh_idx], coarse_mesh[coarse_mesh_idx+1], fine_mesh_len, endpoint=False))[1:]
fine_mesh = fine_mesh_1 + fine_mesh_2
# print(fine_mesh)

In [15]:
isotherm_errors_fine, regularisation_errors_fine = calculate_L_curve(data, fine_mesh)

In [16]:
isotherm_errors_final = np.concatenate((isotherm_errors[:coarse_mesh_idx], isotherm_errors_fine, isotherm_errors[coarse_mesh_idx+1:]))
regularisation_errors_final  = np.concatenate((regularisation_errors[:coarse_mesh_idx], regularisation_errors_fine, regularisation_errors[coarse_mesh_idx+1:]))
final_mesh = coarse_mesh[:coarse_mesh_idx] + fine_mesh + coarse_mesh[coarse_mesh_idx+1:]
plt.plot(isotherm_errors_final, regularisation_errors_final, marker=".")
for i, txt in enumerate(final_mesh):
    plt.annotate(str(round(float(txt), 3)), (isotherm_errors_final[i], regularisation_errors_final[i]))

In [17]:
final_mesh_idx = find_optimal_regularization(isotherm_errors_final, regularisation_errors_final)
print(final_mesh[final_mesh_idx])

0.035662884098438104


In [18]:
from reportParser import parse_all_files
isotherms_data, file_paths = parse_all_files('../data/reports/')

C:\Users\anton\PycharmProjects\isotherm-analyzer\tools\reportParser.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(io.StringIO(value), sep="\s+|\t+|\s+\t+|\t+\s+")
C:\Users\anton\PycharmProjects\isotherm-analyzer\tools\reportParser.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(io.StringIO(value), sep="\s+|\t+|\s+\t+|\t+\s+")
C:\Users\anton\PycharmProjects\isotherm-analyzer\tools\reportParser.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are

In [19]:
isotherm_tmp = Isotherm()
isotherm_tmp.load_kernel()


isotherms = []
for i in range(len(isotherms_data)):
    isotherm = Isotherm()
    
    isotherm.kernel_p_array, isotherm.kernel_a_array, isotherm.kernel_array_of_isotherms = isotherm_tmp.kernel_p_array, isotherm_tmp.kernel_a_array, isotherm_tmp.kernel_array_of_isotherms
    isotherm.kernel_p_array_d, isotherm.kernel_a_array_d, isotherm.kernel_array_of_isotherms_d = isotherm_tmp.kernel_p_array_d, isotherm_tmp.kernel_a_array_d, isotherm_tmp.kernel_array_of_isotherms_d
    
    isotherm.name = file_paths[i]
    isotherm.isothermData = isotherms_data[i]["adsorption"]
    isotherm.pressureData = isotherms_data[i]["p_adsorption"]
    isotherm.interpolate_isotherm()
    isotherm.process_kernel()
    isotherms.append(isotherm)
    print(i, "/", len(isotherms_data))

0 / 1011
1 / 1011
2 / 1011
3 / 1011
4 / 1011
5 / 1011
6 / 1011
7 / 1011
8 / 1011
9 / 1011
10 / 1011
11 / 1011
12 / 1011
13 / 1011
14 / 1011
15 / 1011
16 / 1011
17 / 1011
18 / 1011
19 / 1011
20 / 1011
21 / 1011
22 / 1011
23 / 1011
24 / 1011
25 / 1011
26 / 1011
27 / 1011
28 / 1011
29 / 1011
30 / 1011
31 / 1011
32 / 1011
33 / 1011
34 / 1011
35 / 1011
36 / 1011
37 / 1011
38 / 1011
39 / 1011
40 / 1011
41 / 1011
42 / 1011
43 / 1011
44 / 1011
45 / 1011
46 / 1011
47 / 1011
48 / 1011
49 / 1011
50 / 1011
51 / 1011
52 / 1011
53 / 1011
54 / 1011
55 / 1011
56 / 1011
57 / 1011
58 / 1011
59 / 1011
60 / 1011
61 / 1011
62 / 1011
63 / 1011
64 / 1011
65 / 1011
66 / 1011
67 / 1011
68 / 1011
69 / 1011
70 / 1011
71 / 1011
72 / 1011
73 / 1011
74 / 1011
75 / 1011
76 / 1011
77 / 1011
78 / 1011
79 / 1011
80 / 1011
81 / 1011
82 / 1011
83 / 1011
84 / 1011
85 / 1011
86 / 1011
87 / 1011
88 / 1011
89 / 1011
90 / 1011
91 / 1011
92 / 1011
93 / 1011
94 / 1011
95 / 1011
96 / 1011
97 / 1011
98 / 1011
99 / 1011
100 / 1011

In [20]:
psd_list = []
best_beta_list = []
coarse_mesh =  list(np.logspace(-4, -1, 25))
errors_list = []
for i in range(len(isotherms)):
    isotherm = isotherms[i]
    isotherm_errors, regularisation_errors = calculate_L_curve(isotherm, coarse_mesh)
    coarse_mesh_idx = find_optimal_regularization(isotherm_errors, regularisation_errors)
    
    fine_mesh_len = 20
    fine_mesh_1 = list(np.linspace(coarse_mesh[coarse_mesh_idx - 1], coarse_mesh[coarse_mesh_idx], fine_mesh_len))[1:]
    fine_mesh_2 = list(np.linspace(coarse_mesh[coarse_mesh_idx], coarse_mesh[coarse_mesh_idx+1], fine_mesh_len, endpoint=False))[1:]
    fine_mesh = fine_mesh_1 + fine_mesh_2

    isotherm_errors_fine, regularisation_errors_fine = calculate_L_curve(isotherm, fine_mesh)

    isotherm_errors_final = np.concatenate((isotherm_errors[:coarse_mesh_idx], isotherm_errors_fine, isotherm_errors[coarse_mesh_idx+1:]))
    regularisation_errors_final  = np.concatenate((regularisation_errors[:coarse_mesh_idx], regularisation_errors_fine, regularisation_errors[coarse_mesh_idx+1:]))
    final_mesh = coarse_mesh[:coarse_mesh_idx] + fine_mesh + coarse_mesh[coarse_mesh_idx+1:]

    final_mesh_idx = find_optimal_regularization(isotherm_errors_final, regularisation_errors_final)
    best_beta = final_mesh[final_mesh_idx]
    best_beta_list.append(best_beta)
    isotherm.process_isotherm(best_beta, 0, regularization="tikhonov")
    psd_list.append(isotherm)
    errors_list.append(calculate_error(isotherm)* 100)
    print(i, best_beta, errors_list[i])

0 0.03320480766285996 10.438638983478656
1 0.03717376278124928 4.59439232973413
2 0.018672435543823328 0.67456271826589
3 0.025690996942883362 1.8450060345944528
4 0.02727280485189681 1.8689051917365151
5 0.03584152609815357 1.2739635762113493
6 0.015910128930386708 0.7118179181225065
7 0.026877352874643448 1.3044181052948487
8 0.024504641011123277 0.4998572926835806
9 0.018672435543823328 0.47567055497113797
10 0.03772886139920583 2.270225294919436
11 0.018375888396011963 0.7243025275430619
12 0.01778279410038923 0.5840579678940121
13 0.03828396001716238 2.62415957565361
14 0.01333521432163324 0.521249711568156
15 0.03828396001716238 2.0929739773181324
16 0.014891867244197837 0.9670567259942052
17 0.0173146278078886 0.5918890843717267
18 0.01355759331057104 1.6046350451425302
19 0.0342594950369774 5.362827000690349
20 0.025295544965630003 5.331460820620613
21 0.035314182411094845 1.8616348414298451
22 0.035314182411094845 5.00063934911872
23 0.03828396001716238 1.9054510770174318
24 0

In [34]:
plt.hist(best_beta_list, bins=20)
plt.show()

In [35]:
print(np.std(np.array(best_beta_list)))
print(np.mean(np.array(best_beta_list)))

0.01029914594814088
0.018313845517216324


In [ ]:
plt.scatter(best_beta_list, errors_list)
for i in range(len(best_beta_list)):
    plt.annotate(str(i), (best_beta_list[i], errors_list[i]))
plt.show()

In [21]:
import pandas as pd 
names_list = [isotherm.name for isotherm in psd_list]

In [22]:
df = pd.DataFrame((np.array(names_list), np.array(errors_list), np.array(best_beta_list)))

In [23]:
df.T.to_csv("tyhonov.csv")